In [1]:
import sys
import torch
import numpy as np
import torch
import traci
from sumolib import checkBinary
import torch
import sys
import os
from utils import *
import copy
import gym
import random
from Models import GRUModel
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch.nn.functional as F
import pandas as pd
import math

if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

# sumoBinary = checkBinary('sumo-gui')
sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
SUMO_HOME found
Using device: cuda


In [2]:
class Knowledges:
    def __init__(self):
        self.knowledges = {}
        self.delays = {}
    
    def add_observations(self, vehicles, observed_vehicles):
        for vehicle, visibility in zip(vehicles, observed_vehicles):
            if vehicle not in self.knowledges:
                self.knowledges[vehicle] = []
                self.delays[vehicle] = 0
            self.knowledges[vehicle].append(int(visibility))
            if visibility == 0:
                self.delays[vehicle] += 1
            else:
                self.delays[vehicle] = 0
    
    def merge_knowledges(self, new_knowledges, new_delays):
        prev_missing, prev_delay, _, _ = self.evaluate_knowledge()
        for vehicle, visibility in new_knowledges.items():
            if vehicle not in self.knowledges:
                self.knowledges[vehicle] = copy.deepcopy(visibility)
                self.delays[vehicle] = new_delays[vehicle]
            else:
                for i in range(1, len(self.knowledges[vehicle])+1):
                    if i > len(visibility):
                        break
                    self.knowledges[vehicle][-i] = visibility[-i] | self.knowledges[vehicle][-i]
                self.delays[vehicle] = min(self.delays[vehicle], new_delays[vehicle])
        new_missing, new_delay, _, _ = self.evaluate_knowledge()
        return copy.deepcopy(self.knowledges), copy.deepcopy(self.delays), prev_missing - new_missing, prev_delay - new_delay

    def get_knowledges(self):
        return copy.deepcopy(self.knowledges)
    
    def get_delays(self):
        return copy.deepcopy(self.delays)
    
    def evaluate_knowledge(self, large_delay_threshold=10):
        observed = 0
        delay = 0
        large_delay = 0
        num_vehicles = len(self.knowledges)
        whole_missing = 0
        total = 0
        for vehicle, visibility in self.knowledges.items():
            observed += sum(visibility) 
            total += len(visibility)
            delay += self.delays[vehicle]
            if self.delays[vehicle] >= large_delay_threshold or sum(visibility) <= 3:
                large_delay += 1
            if sum(visibility) == 0:
                whole_missing += len(visibility)
        return 1-(observed / total), delay / num_vehicles, large_delay / num_vehicles, whole_missing / total

class Vehicle:
    def __init__(self):
        self.sent = {}
        self.received = 0

    def step(self):
        self.received = 0
        for vehicle, lag in self.sent.items():
            self.sent[vehicle] += 1
        self.sent = {vehicle: lag for vehicle, lag in self.sent.items() if lag < 10}
    
    def receive(self):
        self.received += 1

    # lag: the time since the last communication with a neighbor
    def select(self, neighbors):
        max_lag = 0
        selected = None
        for neighbor in neighbors:
            if neighbor not in self.sent:
                selected = neighbor
                break
            if self.sent[neighbor] > max_lag:
                max_lag = self.sent[neighbor]
                selected = neighbor
        return selected
    
    def send(self, selected_neighbor):
        self.sent[selected_neighbor] = 0

def connectivity(xs,ys, threshold=800):
    xs = torch.tensor(xs, dtype=torch.float32).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).view(-1,1)
    intervehicle_distances = torch.sqrt((xs - xs.t())**2 + (ys - ys.t())**2)
    if threshold is not None:
        # make the distances 1 if less than the threshold, 0 otherwise
        connectivity = torch.where(intervehicle_distances < threshold, torch.ones_like(intervehicle_distances), torch.zeros_like(intervehicle_distances))
    connectivity = connectivity - torch.diag(torch.diag(connectivity))
    return connectivity, xs, ys

In [3]:
class Incomplete_Routing_Gym(gym.Env):
    def __init__(self, sumoCmd, model, max_steps=1100, n_nodes=57, max_routing_steps=30, max_n_neighbors=6):
        self.sumoCmd = sumoCmd
        self.step_counter = 0
        self.max_steps = max_steps
        self.n_nodes = n_nodes
        self.start_node = None
        self.end_node = None
        self.current_node = None
        self.node_features = None
        self.adj_matrix = None
        self.edge_index = None
        self.hop_thresh = None
        self.routing_done = False
        self.routing_steps = 0
        self.min_n_hops = None
        self.end_node_indicator = torch.zeros(n_nodes)
        self.max_routing_steps = max_routing_steps
        self.n_hop_matrix = None
        self.neighbors_indicator = None
        self.action_space = None
        self.to_remove_indices = None
        self.prunned_adj_matrix = None
        self.prunned_n_hop_matrix = None
        self.state = None
        self.max_n_neighbors = max_n_neighbors
        self.ids = None
        self.vehicle_knowledges = {}
        self.vehicle_records = {}
        self.GRUModel = model
        self.GRUModel.eval()

        self.cached_states = {}

        self.xs = None
        self.ys = None
        self.start_x = None
        self.start_y = None
        self.end_x = None
        self.end_y = None

        self.total_missing_ratio = 0
        self.total_error = 0
        self.total_num_graphs = 0

        self.delay_distribution = []

        self.observed_adj_matrix = None
        self.observed_n_hop_matrix = None

        self.trajectory_dict = {}


        self.load_all_trajectory()

    

    def load_all_trajectory(self):
        self.checkpoints_dict = get_planned_path()
        checkpoints = list(self.checkpoints_dict.values())
        checkpoints = torch.tensor(checkpoints).float() / 10
        position_df = pd.read_csv('trajectory_time.csv')
        position_df.set_index('Unnamed: 0', inplace=True)
        position_array = position_df.to_numpy()
        sequence_length = position_df.shape[1] // 2
        tensor_list = []

        for row in position_array:
            reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
            tensor_list.append(reshaped_tensor)

        all_trajectories_tensor = torch.stack(tensor_list).float() / 10

        next_checkpoint = torch.zeros_like(all_trajectories_tensor)
        next_next_checkpoint = torch.zeros_like(all_trajectories_tensor)
        checkpoints_pad_1 = F.pad(checkpoints, (0, 0, 1, 0))
        for i in range(all_trajectories_tensor.shape[1]):
            _, min_indices = project_to_nearest(all_trajectories_tensor[:, i], checkpoints)
            next_checkpoint[:, i] = checkpoints_pad_1[range(checkpoints.shape[0]), min_indices+1]
            next_next_checkpoint[:, i] = checkpoints_pad_1[range(checkpoints.shape[0]), min_indices+2]

        all_trajectories_tensor = torch.cat((all_trajectories_tensor, next_checkpoint, next_next_checkpoint), dim=2)

        # convert all_trajectories_tensor to a dictionary, where keys are the vehicle ids
        for i, n in enumerate(position_df.index):
            self.trajectory_dict[str(n)] = all_trajectories_tensor[i]
            self.checkpoints_dict[str(n)] = checkpoints[i]

    def reset(self):
        try:
            traci.close()
        except:
            pass
        traci.start(self.sumoCmd)
        self.step_counter = traci.simulation.getTime()

        while self.step_counter < 400:
            self.simstep_with_sync()


    def node_pruning(self):
        self.prunned_adj_matrix = copy.deepcopy(self.observed_adj_matrix)
        self.prunned_n_hop_matrix = copy.deepcopy(self.observed_n_hop_matrix)
        neighbor_indices = np.where(self.observed_adj_matrix[self.current_node] == 1)[0]
        if len(neighbor_indices) >= self.max_n_neighbors:
            two_hop_neighbours_indices = np.where(self.observed_n_hop_matrix[self.current_node] == 2)[0]
            two_hop_neighbours_mask = (self.observed_n_hop_matrix[self.current_node] == 2).type(torch.int)
            # direct neighbours connectivities with two hop neighbours
            neighbour_dict = {}
            for neighbour_index in neighbor_indices:
                neighbour_dict[neighbour_index] = two_hop_neighbours_indices[np.where(self.observed_adj_matrix[neighbour_index][two_hop_neighbours_indices] == 1)[0]]
            # sort by the number of two hop neighbours
            neighbour_dict = dict(sorted(neighbour_dict.items(), key=lambda item: len(item[1]), reverse=True))

            self.to_remove_indices = []
            action_space = 0
            for neighbour_index, two_hop_neighbours_indices in neighbour_dict.items():
                mask_sum_before = torch.sum(two_hop_neighbours_mask)
                two_hop_neighbours_mask[two_hop_neighbours_indices] = 0
                mask_sum_after = torch.sum(two_hop_neighbours_mask)
                if mask_sum_after < mask_sum_before:
                    action_space += 1
                else:
                    self.to_remove_indices.append(neighbour_index)
            if action_space < self.max_n_neighbors:
                self.to_remove_indices = random.sample(self.to_remove_indices, len(self.to_remove_indices) - (self.max_n_neighbors - action_space))
            self.prunned_adj_matrix[self.to_remove_indices, :] = 0
            self.prunned_adj_matrix[:, self.to_remove_indices] = 0
            self.prunned_n_hop_matrix[self.to_remove_indices, :] = -100
            self.prunned_n_hop_matrix[:, self.to_remove_indices] = -100
        self.prunned_n_hop_matrix = self.prunned_n_hop_matrix - torch.diag(torch.diag(self.prunned_n_hop_matrix))
        self.prunned_adj_matrix = self.prunned_adj_matrix - torch.diag(torch.diag(self.prunned_adj_matrix))


    def next_episode(self, refresh=False):
        self.routing_done = False
        self.routing_steps = 0
        if not refresh:
            self.simstep_with_sync()
            self.adj_matrix = F.pad(self.adj_matrix, (0, self.n_nodes - self.adj_matrix.size(0), 
                                                  0, self.n_nodes - self.adj_matrix.size(1)), "constant", 0)
            self.n_hop_matrix = F.pad(self.n_hop_matrix, (0, self.n_nodes - self.n_hop_matrix.size(0), 
                                                      0, self.n_nodes - self.n_hop_matrix.size(1)), "constant", -100)
        self.select_start_end_nodes()
        self.current_node = self.start_node
        self.get_state()
        return copy.deepcopy(self.state)

    def simstep_with_sync(self):
        traci.simulationStep()
        self.cached_states = {}
        self.step_counter = int(traci.simulation.getTime())
        self.adj_matrix, self.xs, self.ys = intervehicleConnectivity_xs_ys(800)
        self.n_hop_matrix = bfs_distance(self.adj_matrix)
        self.ids = traci.vehicle.getIDList()
        action_spaces = self.adj_matrix - torch.diag(torch.diag(self.adj_matrix))

        for i, vehicle in enumerate(self.ids):
            if vehicle not in self.vehicle_knowledges:
                self.vehicle_knowledges[vehicle] = Knowledges()
            if vehicle not in self.vehicle_records:
                self.vehicle_records[vehicle] = Vehicle()
            self.vehicle_knowledges[vehicle].add_observations(self.ids, self.adj_matrix[i])
        
        for i, vehicle in enumerate(self.ids):
            non_zero_indices = torch.where(action_spaces[i] == 1)[0]
            neighbors = [self.ids[j] for j in non_zero_indices] 
            if len(neighbors) > 0:
                select_neighbour = self.vehicle_records[vehicle].select(neighbors)
                if select_neighbour is None:
                    continue
                self.vehicle_records[vehicle].send(select_neighbour)
                self.vehicle_knowledges[select_neighbour].merge_knowledges(self.vehicle_knowledges[vehicle].get_knowledges(), self.vehicle_knowledges[vehicle].get_delays())

                self.vehicle_records[select_neighbour].send(vehicle)
                self.vehicle_knowledges[vehicle].merge_knowledges(self.vehicle_knowledges[select_neighbour].get_knowledges(), self.vehicle_knowledges[select_neighbour].get_delays())
            
        for vehicle in self.vehicle_records.values():
            vehicle.step()

        
    def select_start_end_nodes(self):
        self.hop_thresh = min(self.n_hop_matrix.max(), 5)
        starts, ends = torch.where(self.hop_thresh == self.n_hop_matrix)
        starts = starts.tolist()
        ends = ends.tolist()
        self.start_node, self.end_node = random.choice(list(zip(starts, ends)))
        # minimal number of hops between start and end nodes
        self.min_n_hops = self.n_hop_matrix[self.start_node, self.end_node]

        self.start_x = self.xs[self.start_node]
        self.start_y = self.ys[self.start_node]
        self.end_x = self.xs[self.end_node]
        self.end_y = self.ys[self.end_node]

    
    def get_state(self):
        if self.current_node in self.cached_states:
            self.state = copy.deepcopy(self.cached_states[self.current_node])
        self.compute_state()
        self.cached_states[self.current_node] = copy.deepcopy(self.state)
    

    def compute_state(self):
        current_vehicle_knowledge = self.vehicle_knowledges[self.ids[self.current_node]].get_knowledges()
        current_vehicle_delay = self.vehicle_knowledges[self.ids[self.current_node]].get_delays()
        observed_vehicle_xs = []
        observed_vehicle_ys = []
        for vehicle in self.ids:
            if current_vehicle_delay[vehicle] == 0:
                observed_vehicle_xs.append(self.xs[self.ids.index(vehicle)])
                observed_vehicle_ys.append(self.ys[self.ids.index(vehicle)])
            elif current_vehicle_delay[vehicle] >= 10 or sum(current_vehicle_knowledge[vehicle]) <= 3:
                observed_vehicle_xs.append(0)
                observed_vehicle_ys.append(0)
            else:
                x, y = self.estimate_vehicle_positions(vehicle, current_vehicle_knowledge[vehicle], current_vehicle_delay[vehicle])
                observed_vehicle_xs.append(x)
                observed_vehicle_ys.append(y)
        
        self.observed_adj_matrix, xs, ys = connectivity(observed_vehicle_xs, observed_vehicle_ys)
        # set connectivity to 0 if the vehicle is not observed
        not_observed_indices = torch.where(xs == -1)[0]
        self.observed_adj_matrix[not_observed_indices, :] = 0
        self.observed_adj_matrix[:, not_observed_indices] = 0

        self.observed_adj_matrix = F.pad(self.observed_adj_matrix, (0, self.n_nodes - self.observed_adj_matrix.size(0), 
                                                  0, self.n_nodes - self.observed_adj_matrix.size(1)), "constant", 0)
        # set the neighbouring connectivity to the true connectivity
        non_neighbour_indices = torch.where(self.adj_matrix[self.current_node] == 0)[0]
        self.observed_adj_matrix[self.current_node, non_neighbour_indices] = 0
        self.observed_adj_matrix[non_neighbour_indices, self.current_node] = 0

        norm_xs = (xs - self.end_x) / (self.start_x - self.end_x)
        norm_ys = (ys - self.end_y) / (self.start_y - self.end_y)
        # set the norm_xs and norm_ys to 0 if the vehicle is not observed
        norm_xs[not_observed_indices] = 0
        norm_ys[not_observed_indices] = 0
        # padding the matrices to the same size
        norm_xs = F.pad(norm_xs, (0, 0, 0, self.n_nodes - norm_xs.size(0)), "constant", 0)
        norm_ys = F.pad(norm_ys, (0, 0, 0, self.n_nodes - norm_ys.size(0)), "constant", 0)
        self.observed_n_hop_matrix = bfs_distance(self.observed_adj_matrix)
        self.node_pruning()
        edge_index = dense_to_sparse(self.prunned_adj_matrix)[0]

        curr_node_indicator = torch.zeros(self.n_nodes)
        curr_node_indicator[self.current_node] = 1
        distances = self.prunned_n_hop_matrix[self.end_node]
        distances[distances == -100] = 8
        distances[distances > 7] = 7
        one_hot_distances = F.one_hot(distances.long(), num_classes=8).type(torch.float32)
        neighbour_indicator = self.prunned_adj_matrix[self.current_node]
        node_features = torch.cat((one_hot_distances, self.end_node_indicator.unsqueeze(1), 
                                   curr_node_indicator.unsqueeze(1), neighbour_indicator.unsqueeze(1), 
                                   norm_xs, norm_ys), dim=1).to(device)
        self.state = Data(x=node_features, edge_index=edge_index)


    def act(self, neighbor_index):
        self.routing_steps += 1
        neighbors = torch.where(self.prunned_adj_matrix[self.current_node] == 1)[0]
        valid_action_size = len(neighbors)
        if valid_action_size <= neighbor_index:
            self.routing_done = self.routing_steps >= self.max_routing_steps
            if self.routing_done:
                return self.state, torch.tensor(-1).to(device), self.routing_done
            return self.state, torch.tensor(-0.15).to(device), self.routing_done
        else:
            next_hop = neighbors[neighbor_index]
            reward = self.compute_reward(next_hop)
            self.current_node = next_hop
            self.get_state()
            return self.state, torch.tensor(reward).to(device), self.routing_done

    
    def estimate_vehicle_positions(self, vehicle, observation_history, delay):
        last_seen_time = int(self.step_counter - delay)
        first_one_index = int(observation_history.index(1))
        sequence_length = min(len(observation_history) - first_one_index, 20)
        # these last seen value will be used to calibrate the position of the vehicle
        # we set the last seen value to 50, 50, as the autoregressive model is trained on the normalized values
        last_seen_x = self.trajectory_dict[vehicle][last_seen_time][0].item()
        last_seen_y = self.trajectory_dict[vehicle][last_seen_time][1].item()
        modif = (torch.tensor([last_seen_x, last_seen_y])-torch.tensor([50, 50])).repeat(1, 3)

        masks = torch.tensor(observation_history).unsqueeze(0).unsqueeze(2).repeat(1, 1, 6)

        paths = self.checkpoints_dict[vehicle].unsqueeze(0)
        inputs = copy.deepcopy(self.trajectory_dict[vehicle][self.step_counter-sequence_length+1:self.step_counter+1].unsqueeze(0))
        inputs -= modif
        masks = masks[:, -sequence_length:, :]
        modified_x, modified_y = self.GRU_inference(inputs.to(device), masks.to(device), paths.to(device))
        x, y = (modified_x + modif[0][0]).item() * 10, (modified_y + modif[0][1]).item() * 10
        return x, y
    

    def GRU_inference(self, inputs, masks, paths):
        with torch.no_grad():
            hidden = None
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = self.GRUModel(current_input, hidden)
                projection_with_checkpoints = project_to_nearest_with_checkpoints(prediction, paths)
                current_input = (projection_with_checkpoints * (1-masks[:, t, :]) + inputs[:, t, :] * (masks[:, t, :])).unsqueeze(1)
        return current_input[0,0,0], current_input[0,0,1]
    

    def compute_reward(self, next_hop):
        if self.routing_steps >= self.max_routing_steps:
            self.routing_done = True
            return -1
        elif self.adj_matrix[self.current_node, self.end_node] == 1:
            self.routing_done = True
            return (self.min_n_hops / self.routing_steps)
        elif self.n_hop_matrix[self.current_node, self.end_node] > self.n_hop_matrix[next_hop, self.end_node]:
            return 0.1
        else:
            return -0.15


    def get_action_mask(self):
        action_mask = copy.deepcopy(self.prunned_adj_matrix[self.current_node])
        action_mask = F.pad(action_mask, (0, self.n_nodes - action_mask.size(0)), "constant", 0).to(device)
        return action_mask


    def sim_done(self):
        """
        function: get the done state of simulation.
        """
        return not (shouldContinueSim() and self.step_counter < self.max_steps)


# Test the Greedy BFS on Env

In [4]:
model = GRUModel(input_size=6, hidden_size=256, num_layers=2, output_size=2).to(device)
# load the model
model.load_state_dict(torch.load('models/gru_trajectory_prediction.pth'))
model.eval()
env = Incomplete_Routing_Gym(sumoCmd, model)
env.reset()
sim_done = False
iteration = 10
no_direct_path = 0
total_tasks = 0
total_reward = 0

step_counter = 0
while not sim_done:
    state = env.next_episode()
    step_counter = env.step_counter
    if step_counter % 10 == 0:
        print("Episode: ", step_counter)
    if step_counter % 20 == 0:
        print("No direct path: ", no_direct_path)
        print("Total tasks: ", total_tasks)
        print("Total reward: ", total_reward)
    sim_done = env.sim_done()
    for i in range(iteration):
        state = env.next_episode(refresh=True)
        total_tasks += 1
 
        routing_done = False
        episode_reward = 0
        skip = False
        while not routing_done:
            action_mask = env.get_action_mask()
            action_indices = torch.where(action_mask == 1)[0]

            one_hot_distance_to_end = state.x[:, :8]
            distance_to_end = torch.argmax(one_hot_distance_to_end, dim=1)
            current_node = env.current_node
            current_distance = distance_to_end[current_node]

            masked_distance_to_end = distance_to_end * action_mask + 8 * (1 - action_mask)
            closer_to_end = torch.where(masked_distance_to_end < current_distance)[0]
            if len(closer_to_end) == 0:
                skip = True
                no_direct_path += 1
                break
            selected_action = random.choice(closer_to_end)
            selected_action_index = int(torch.where(action_indices == selected_action)[0])
            state, reward, routing_done = env.act(selected_action_index)
            
            episode_reward += reward.item()
        if not skip:
            total_reward += episode_reward

 Retrying in 1 seconds
***Starting server on port 48277 ***
Loading net-file from './config/osm.net.xml.gz' ... done (117ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 1557.00
Reason: TraCI requested termination.
Performance: 
 Duration: 0.38s
 TraCI-Duration: 0.23s
 Real time factor: 4097.37
 UPS: 121136.842105
Vehicles: 
 Inserted: 89
 Running: 0
 Waiting: 0
Statistics (avg of 89):
 RouteLength: 4885.02
 Speed: 9.58
 Duration: 517.21
 WaitingTime: 17.72
 TimeLoss: 70.09
 DepartDelay: 0.47

 Retrying in 1 seconds
***Starting server on port 42045 ***
Loading net-file from './config/osm.net.xml.gz' ... done (103ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.


/tmp/ipykernel_5420/2162613853.py:273: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return self.state, torch.tensor(reward).to(device), self.routing_done


In [5]:
print("total_reward: ", total_reward)
print("total_tasks: ", total_tasks)
print("average_reward: ", total_reward / (total_tasks - no_direct_path))
print("no_direct_path: ", no_direct_path)

total_reward:  56.07121231406927
total_tasks:  50
average_reward:  1.367590544245592
no_direct_path:  9
